In [52]:
%matplotlib inline
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import hvplot.pandas
import numpy as np
import seaborn as sns
from pathlib import Path

In [53]:
# Define function to create Pickle File
def get_api_dataframe():
    # Set Alpaca API key and secret

    load_dotenv()
    api_key= os.getenv("ALPACA_API_KEY")
    sec_key= os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object

    alpaca = tradeapi.REST(
        api_key,
        sec_key,
        api_version="v2"
    )
# Get 3 years worth of historical data for Alcohol stocks, Cannabis Stocks, and S&P 500

    start_date = pd.Timestamp('2019-04-20', tz='America/New_York').isoformat()
    end_date = pd.Timestamp('2022-04-20', tz='America/New_York').isoformat()

    #tickers = ["BUD", "DEO", "SAM", "TAP", "STZ","VFF","JAZZ","CRON", "TLRY","CGC", "SPY"]
    tickers = ["BUD"]

# Set timeframe to "1Day" for Alpaca API
    timeframe = "1Day"

# Get current closing prices for All Stocks

    bud_stock_data = alpaca.get_bars(
        tickers,
        timeframe,
        start = start_date,
        end = end_date
    ).df

#Set index to date
    bud_stock_data.index = bud_stock_data.index.date
    return bud_stock_data

In [54]:
# Check if file exists
# If file does not exist then load api and save file
# else read data from pikl

bud_data_file = Path("bud_data.pkl")
if bud_data_file.is_file():
    print ("API data file exists")
    bud_stock_data = pd.read_pickle(bud_data_file)
else:
    print ("API file does not exist") 
    bud_stock_data = get_api_dataframe()
    bud_stock_data.to_pickle(bud_data_file)

#View dataframe
bud_stock_data.head(-5)

API data file exists


,open,high,low,close,volume,trade_count,vwap,symbol
2019-04-22,89.69,90.470,89.5800,89.60,970834,9479,90.000447,BUD
2019-04-23,88.56,89.790,88.5000,89.25,755451,7907,89.242818,BUD
2019-04-24,89.02,89.130,87.8200,88.21,1038366,11509,88.324096,BUD
2019-04-25,86.89,87.020,86.0000,86.64,1341678,13124,86.467163,BUD
2019-04-26,87.38,87.625,86.8300,87.58,920430,9005,87.292918,BUD
...,...,...,...,...,...,...,...,...
2022-04-06,59.65,60.050,59.4200,59.75,1400313,16245,59.756942,BUD
2022-04-07,58.37,58.650,57.9900,58.47,2025008,17382,58.353684,BUD
2022-04-08,58.46,59.240,58.2900,58.78,964847,11297,58.850033,BUD
2022-04-11,58.42,59.020,58.3519,58.49,1458085,15172,58.643236,BUD


In [55]:
# Calculate the daily returns using the closing prices and the pct_change function
bud_stock_data["actual_returns"] = bud_stock_data["close"].pct_change()

# Display sample data
bud_stock_data.head()

,open,high,low,close,volume,trade_count,vwap,symbol,actual_returns
2019-04-22,89.69,90.470,89.58,89.60,970834,9479,90.000447,BUD,NaN
2019-04-23,88.56,89.790,88.50,89.25,755451,7907,89.242818,BUD,-0.003906
2019-04-24,89.02,89.130,87.82,88.21,1038366,11509,88.324096,BUD,-0.011653
2019-04-25,86.89,87.020,86.00,86.64,1341678,13124,86.467163,BUD,-0.017798
2019-04-26,87.38,87.625,86.83,87.58,920430,9005,87.292918,BUD,0.010849


In [56]:
# Drop all NaN values from the DataFrame
bud_stock_data = bud_stock_data.dropna()

# Review the DataFrame
display(bud_stock_data.head())
display(bud_stock_data.tail())

,open,high,low,close,volume,trade_count,vwap,symbol,actual_returns
2019-04-23,88.56,89.790,88.50,89.25,755451,7907,89.242818,BUD,-0.003906
2019-04-24,89.02,89.130,87.82,88.21,1038366,11509,88.324096,BUD,-0.011653
2019-04-25,86.89,87.020,86.00,86.64,1341678,13124,86.467163,BUD,-0.017798
2019-04-26,87.38,87.625,86.83,87.58,920430,9005,87.292918,BUD,0.010849
2019-04-29,87.35,88.370,87.27,88.01,840389,8728,87.936270,BUD,0.004910


,open,high,low,close,volume,trade_count,vwap,symbol,actual_returns
2022-04-13,57.92,58.72,57.90,58.58,964169,11290,58.388346,BUD,0.010871
2022-04-14,58.81,59.00,58.49,58.78,960973,10996,58.798913,BUD,0.003414
2022-04-18,58.65,59.01,58.46,58.72,734730,10193,58.727354,BUD,-0.001021
2022-04-19,58.45,59.25,58.44,59.19,1321247,15500,58.935246,BUD,0.008004
2022-04-20,60.99,61.99,60.97,61.49,1401489,17526,61.522348,BUD,0.038858


In [57]:
#Create Feature Set and Window Sizes

short_window = 15

# Create an SMA that uses short_window, and assign it to a new column named “sma_fast”
bud_stock_data["sma_fast"] = bud_stock_data["close"].rolling(window=short_window).mean()


# Define a window size 
long_window = 50

# Create an SMA that uses long_window, and assign it to a new columns named “sma_slow”
bud_stock_data["sma_slow"] = bud_stock_data["close"].rolling(window=long_window).mean()

In [58]:
# Drop the NaNs using dropna()
bud_stock_data = bud_stock_data.dropna()

In [59]:
bud_stock_data["sma_slow_lag"] = bud_stock_data["sma_slow"].shift(1)
bud_stock_data["sma_fast_lag"] = bud_stock_data["sma_fast"].shift(1)
bud_stock_data['l1'] = bud_stock_data['actual_returns'].shift(1)
bud_stock_data = bud_stock_data.dropna()
bud_stock_data

,open,high,low,close,volume,trade_count,vwap,symbol,actual_returns,sma_fast,sma_slow,sma_slow_lag,sma_fast_lag,l1
2019-07-03,93.31,94.16,92.870,93.93,1552523,14990,93.637161,BUD,0.036183,88.240667,85.6310,85.5374,87.765333,0.025105
2019-07-05,92.83,92.89,91.960,92.71,1112826,12700,92.458643,BUD,-0.012988,88.692667,85.7210,85.6310,88.240667,0.036183
2019-07-08,92.45,92.63,91.555,91.68,936305,10364,92.017916,BUD,-0.011110,89.192667,85.8218,85.7210,88.692667,-0.012988
2019-07-09,91.89,92.11,91.245,91.51,1717665,16980,91.602596,BUD,-0.001854,89.672000,85.9004,85.8218,89.192667,-0.011110
2019-07-10,91.80,91.88,90.610,91.35,1657277,14151,91.158102,BUD,-0.001748,89.970667,85.9672,85.9004,89.672000,-0.001854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-13,57.92,58.72,57.900,58.58,964169,11290,58.388346,BUD,0.010871,59.872667,60.1298,60.2316,59.881333,-0.009232
2022-04-14,58.81,59.00,58.490,58.78,960973,10996,58.798913,BUD,0.003414,59.849333,60.0282,60.1298,59.872667,0.010871
2022-04-18,58.65,59.01,58.460,58.72,734730,10193,58.727354,BUD,-0.001021,59.796667,59.9676,60.0282,59.849333,0.003414
2022-04-19,58.45,59.25,58.440,59.19,1321247,15500,58.935246,BUD,0.008004,59.721333,59.9284,59.9676,59.796667,-0.001021


In [60]:
# Assign a copy of the `sma_fast` and `sma_slow` columns to a new DataFrame called `X`
#X = bud_stock_data[["sma_fast", "sma_slow", "close"]].shift(1).dropna().copy()
X = bud_stock_data[["sma_fast_lag", "sma_slow_lag", "l1"]].dropna().copy()
# Display sample data
display(X.head())
display(X.tail())

,sma_fast_lag,sma_slow_lag,l1
2019-07-03,87.765333,85.5374,0.025105
2019-07-05,88.240667,85.6310,0.036183
2019-07-08,88.692667,85.7210,-0.012988
2019-07-09,89.192667,85.8218,-0.011110
2019-07-10,89.672000,85.9004,-0.001854


,sma_fast_lag,sma_slow_lag,l1
2022-04-13,59.881333,60.2316,-0.009232
2022-04-14,59.872667,60.1298,0.010871
2022-04-18,59.849333,60.0282,0.003414
2022-04-19,59.796667,59.9676,-0.001021
2022-04-20,59.721333,59.9284,0.008004


In [61]:
#Create Target Set
# Create a new column in the `df_stock_data` called "signal" setting its value to zero.
bud_stock_data["signal"] = 0.0

# Create the signal to buy
bud_stock_data.loc[(bud_stock_data["actual_returns"] >= 0), "signal"] = 1

# Create the signal to sell
bud_stock_data.loc[(bud_stock_data["actual_returns"] < 0), "signal"] = -1

In [62]:
# Copy the new "signal" column to a new Series called `y`.
y = bud_stock_data["signal"].copy()

In [63]:
#Split the Data in Training and Test Sets
# Import required libraries
from pandas.tseries.offsets import DateOffset

# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2019-07-03


In [64]:
# Select the ending period for the training data with an offset of 18 months
training_end = X.index.min() + DateOffset(months=24)

# Display the training end date
print(training_end)

2021-07-03 00:00:00


In [65]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

c:\Users\themi\.conda\envs\alpacaenv\lib\site-packages\pandas\core\algorithms.py:1601: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  return arr.searchsorted(value, side=side, sorter=sorter)


In [66]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [67]:
#Standardize the Data
# Import required libraries
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler instance
scaler = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [68]:
X_test.shape

(201, 3)

In [69]:
X_train.shape

(505, 3)

In [70]:
# Create trading strategy
# Import the SVM model
from sklearn import svm
from sklearn.metrics import classification_report

In [71]:
# Create the classifier model
svm_model = svm.SVC()
 
# Fit the model to the data using X_train_scaled and y_train
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
training_signal_predictions[:10]

array([-1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.])

In [79]:
# Evaluate the model using a classification report
training_report = classification_report(y_train, training_signal_predictions)

# Display the report
print(training_report, "BUD - Training Report")

              precision    recall  f1-score   support

        -1.0       0.60      0.48      0.54       251
         1.0       0.57      0.69      0.62       254

    accuracy                           0.58       505
   macro avg       0.59      0.58      0.58       505
weighted avg       0.59      0.58      0.58       505
 BUD - Training Report


In [73]:
#Backtesting a Machine Learning Trading Algorithm
# Use the trained model to predict the trading signals for the testing data.
testing_signal_predictions = svm_model.predict(X_test_scaled)

In [81]:
# Evaluate the model's ability to predict the trading signal for the testing data
testing_report = classification_report(y_test, testing_signal_predictions)

# Display the report
print(testing_report, "BUD - Testing Report")

              precision    recall  f1-score   support

        -1.0       0.63      0.20      0.31       108
         1.0       0.48      0.86      0.62        93

    accuracy                           0.51       201
   macro avg       0.56      0.53      0.46       201
weighted avg       0.56      0.51      0.45       201
 BUD - Testing Report


In [75]:
# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

predictions_df["predicted_signal"] = testing_signal_predictions

predictions_df["actual_returns"] = bud_stock_data["actual_returns"]

predictions_df["trading_algorithm_returns"] = (
    predictions_df["actual_returns"] * predictions_df["predicted_signal"]
)

# Review the DataFrame
predictions_df.head()

,predicted_signal,actual_returns,trading_algorithm_returns
2021-07-06,-1.0,-0.015949,0.015949
2021-07-07,-1.0,0.006255,-0.006255
2021-07-08,-1.0,-0.016530,0.016530
2021-07-09,-1.0,0.003879,-0.003879
2021-07-12,-1.0,-0.003864,0.003864


In [77]:
# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
(1 + predictions_df[["actual_returns", "trading_algorithm_returns"]]).cumprod().hvplot(
    xlabel = "Date",
    ylabel = "Returns",
    label = "Anheuser-Busch (BUD) - Long Algo"
)

:NdOverlay   [Variable]
   :Curve   [index]   (value)